## matlabengine
Load in different .m files from matlab using matlab API

In [98]:
import netCDF4 as nc
from netCDF4 import Dataset
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# gsw oceanic toolbox: http://www.teos-10.org/pubs/Getting_Started.pdf
import gsw
from scipy.io import loadmat

In [99]:
arctic_nc = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/arctic_mmp.nc"
arctic_dataset = Dataset(arctic_nc)
arctic_ds = xr.open_dataset(arctic_nc)
gamma_m = '/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/gamma_GP_from_SP_pt.m'

In [100]:
arctic_ds['pot_temp'] = gsw.conversions.pt0_from_t(arctic_ds.PSAL,arctic_ds.TEMPERATURE,arctic_ds.PRESSURE)

T and S must have MxN dimensions, p can be 1x1, or 1xN or Mx1.

In [116]:
# convert xr data array to numpy array using .values
# np.arrays are supported in matlab engine

T = arctic_ds["pot_temp"].values
# flatten T for the last dimension
T = np.squeeze(T, axis=-1) 
S = arctic_ds["PSAL"].values
p = arctic_ds["PRESSURE"].values
p = np.squeeze(p, axis=-1) 
lat = arctic_ds["LATITUDE"].values
lat = np.squeeze(lat, axis=-1) 
lon = arctic_ds["LONGITUDE"].values
lon = np.squeeze(lon, axis=-1) 
z = arctic_ds["depth"].values

In [141]:
T.shape

(871, 1201)

In [140]:
p.shape

(1201,)

In [123]:
lon.shape

(871,)

In [120]:
S.shape

(871, 1201)

In [121]:
z.shape

(1201,)

In [135]:
[Zmesh, latmesh]=np.meshgrid(z, lat);
[Zmesh, latmesh]=np.meshgrid(z, lon);

In [142]:
latmesh.shape

(871, 1201)

In [137]:
Zmesh.shape

(871, 1201)

In [108]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [138]:
eng.addpath('/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence')

# Set the input arguments in the MATLAB workspace
eng.workspace['SP'] = S
eng.workspace['pt'] = T
eng.workspace['p'] = p
eng.workspace['long'] = lonmesh
eng.workspace['lat'] = latmesh

eng.eval("gamma = gamma_GP_from_SP_pt(SP, pt, p, long, lat)", nargout=0)

# Disconnect from the MATLAB engine
eng.quit()

Error using gamma_GP_from_SP_pt
gamma_GP_from_SP_pt: Inputs array dimensions arguments do not agree



MatlabExecutionError: 
  File /Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/gamma_GP_from_SP_pt.m, line 121, in gamma_GP_from_SP_pt
gamma_GP_from_SP_pt: Inputs array dimensions arguments do not agree


In [115]:
eng.addpath('/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/')
#eng.eval("run('/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/gamma_GP_from_SP_pt.m')", nargout=0)
eng.eval("gamma = gamma_GP_from_SP_pt(S, T, p, long, lat)", nargout=0)

Error using eval
Unrecognized function or variable 'S'.



MatlabExecutionError: Unrecognized function or variable 'S'.


In [112]:
eng.eval("run('/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/gamma_GP_from_SP_pt.m')", nargout=0)
gamma = eng.gamma_GP_from_SP_pt(S, T, p, lonmesh, latmesh, nargout=1)

Error using gamma_GP_from_SP_pt
gamma_GP_from_SP_pt:  Requires five inputs

Error in run (line 91)
evalin('caller', strcat(script, ';'));



MatlabExecutionError: 
  File /Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/gamma_GP_from_SP_pt.m, line 54, in gamma_GP_from_SP_pt

  File /Applications/MATLAB_R2023a.app/toolbox/matlab/lang/run.m, line 91, in run
gamma_GP_from_SP_pt:  Requires five inputs
